# Goldbach Runner

This notebook demonstrates how to create a grading workflow using PyBryt.

In [1]:
import pybryt

This demo has the following directory structure. This notebook, `index.ipynb`, runs PyBryt, `goldbach.ipynb` is the assignment reference implementation, and `submissions` contains notebooks with student code in them.

In [2]:
%%bash
tree

.
├── goldbach-ref.pkl
├── goldbach.ipynb
├── index.ipynb
└── submissions
    ├── 1.ipynb
    ├── 1.py
    ├── 2.ipynb
    ├── 2.py
    ├── 3.ipynb
    ├── 3.py
    ├── 4.ipynb
    ├── 4.py
    ├── 5.ipynb
    ├── 5.py
    ├── 6.ipynb
    └── 6.py

1 directory, 15 files


## Reference Implementations

If you have marked up a reference implementation, like the one in [`median.ipynb`](median.ipynb), you can load this reference using `pybryt.ReferenceImplementation.compile`. Because references are relatively static and can take some time to execute, you can pickle the reference implementations to a file with `pybryt.ReferenceImplementation.dump`.

In [3]:
ref = pybryt.ReferenceImplementation.compile("goldbach.ipynb")
ref.dump("goldbach-ref.pkl")

To load a pickled reference implementation, use `pybryt.ReferenceImplementation.load`:

In [4]:
ref = pybryt.ReferenceImplementation.load("goldbach-ref.pkl")


## Assessing Submissions

To use PyBryt for grading multiple submissions, you can build a reproducible grading pipeline for an arbitrary number of submissions. To grab the submission notebook paths, the cell below uses `glob.glob`.

In [5]:
from glob import glob
subms = glob("submissions/*.py")
subms

['submissions/6.py',
 'submissions/2.py',
 'submissions/3.py',
 'submissions/4.py',
 'submissions/5.py',
 'submissions/1.py']

In [6]:
import os
import IPython.nbformat  as nbf
from pathlib import Path

notebooks = []

for sub in subms:

    code = Path(sub).read_text()
    
    cells = []

    cell = nbf.v4.new_code_cell(source=code)
    cells.append(cell)
    cell = nbf.v4.new_code_cell(source="from sympy.ntheory.generate import primerange\nprimes = set(primerange(50,1000))\ncheck_goldbach_for_num(116, primes)")
    cells.append(cell)

    nb = nbf.v4.new_notebook(cells = cells)

    pre, ext = os.path.splitext(sub)
    notebook = pre + '.ipynb'
    with open(notebook, 'w') as f:
        
            nbf.write(nb, f, 4)
    notebooks.append(notebook)

/Users/chrispyles/Library/Python/3.7/lib/python/site-packages/IPython/nbformat.py:13: ShimWarning: The `IPython.nbformat` package has been deprecated since IPython 4.0. You should import from nbformat instead.
  "You should import from nbformat instead.", ShimWarning)


To use PyBryt to grade a student's submission, a `pybryt.StudentImplementation` must be created from that submission. The constructor takes the path to the notebook as its only positional argument. It is in this step that the student's code is executed, so this cell will need to be rerun whenever changes are made to the submission notebooks.

In [7]:
student_impls = []
for subm in notebooks:
    print(subm)
    student_impls.append(pybryt.StudentImplementation(subm))

student_impls

submissions/6.ipynb
submissions/2.ipynb
submissions/3.ipynb
submissions/4.ipynb
submissions/5.ipynb
submissions/1.ipynb


Once you have created the `pybryt.StudentImplementation` objects, use the `pybryt.StudentImplementation.check` method to run the check of a submission against a reference implementation. This method returns a single `pybryt.ReferenceResult` or a list of them, depending on the argument passed to `check`. In the cell below, the results are collected into a list.

In [8]:
results = []
for si in student_impls:
    results.append(si.check(ref))

results

[ReferenceResult([
   AnnotationResult(satisfied=True, annotation=pybryt.NotAnnotation),
   AnnotationResult(satisfied=True, annotation=pybryt.NotAnnotation),
   AnnotationResult(satisfied=True, annotation=pybryt.NotAnnotation),
   AnnotationResult(satisfied=True, annotation=pybryt.NotAnnotation),
   AnnotationResult(satisfied=True, annotation=pybryt.NotAnnotation)
 ]),
 ReferenceResult([
   AnnotationResult(satisfied=True, annotation=pybryt.NotAnnotation),
   AnnotationResult(satisfied=True, annotation=pybryt.NotAnnotation),
   AnnotationResult(satisfied=True, annotation=pybryt.NotAnnotation),
   AnnotationResult(satisfied=True, annotation=pybryt.NotAnnotation),
   AnnotationResult(satisfied=True, annotation=pybryt.NotAnnotation)
 ]),
 ReferenceResult([
   AnnotationResult(satisfied=True, annotation=pybryt.NotAnnotation),
   AnnotationResult(satisfied=True, annotation=pybryt.NotAnnotation),
   AnnotationResult(satisfied=True, annotation=pybryt.NotAnnotation),
   AnnotationResult(satis

To view the results in a concise manner, the `pybryt.ReferenceResult` class has some helpful instance variables:

In [9]:
from textwrap import indent
for sp, res in zip(subms, results):
    print(f"SUBMISSION: {sp}")
    # res.messages is a list of messages returned by the reference during grading
    messages = "\n".join(res.messages) 
    
    # res.correct is a boolean for whether the reference was satisfied
    message = f"SATISFIED: {res.correct}\nMESSAGES:\n{indent(messages, '  - ')}"
    
    # some pretty-printing
    print(indent(message, "  "))
    print("\n")

SUBMISSION: submissions/6.py
  SATISFIED: True
  MESSAGES:



SUBMISSION: submissions/2.py
  SATISFIED: True
  MESSAGES:



SUBMISSION: submissions/3.py
  SATISFIED: True
  MESSAGES:



SUBMISSION: submissions/4.py
  SATISFIED: True
  MESSAGES:



SUBMISSION: submissions/5.py
  SATISFIED: True
  MESSAGES:



SUBMISSION: submissions/1.py
  SATISFIED: True
  MESSAGES:



